## function auf listen

In [14]:
maxlist :: Ord a => [a] -> Maybe a
maxlist [] = Nothing
maxlist [x] = return x
maxlist (x:xs) = maxlist xs >>= \y -> return $ max x y

maxlist (10:[1..3])

Just 10

## mit applicative

In [16]:
maxlist :: Ord a => [a] -> Maybe a
maxlist [] = Nothing
maxlist [x] = return x
maxlist (x:xs) = max <$> pure x <*> maxlist xs

maxlist (10:[1..3])

Line 4: Redundant <$>
Found:
max <$> pure x
Why not:
pure (max x)

Just 10

## verallgemeinern

In [10]:
import Control.Monad
import Control.Applicative

In [17]:
maxlist :: (MonadPlus m, Ord a) => [a] -> m a
maxlist [] = mzero
maxlist [x] = return x
maxlist (x:xs) = max <$> pure x <*> maxlist xs

maxlist (10:[1..3])

Line 4: Redundant <$>
Found:
max <$> pure x
Why not:
pure (max x)

10

In [22]:
maxlist :: (Alternative f, Ord a) => [a] -> f a
maxlist [] = empty
maxlist [x] = pure x
maxlist (x:xs) = max <$> pure x <*> maxlist xs

maxlist (10:[1..3])

maxlist [[1,2,3,4], [1,1,3,4]]

Line 4: Redundant <$>
Found:
max <$> pure x
Why not:
pure (max x)

10

[1,2,3,4]

## unterscheid zw monade und monoid

- monoid sind typen mit kind *              zb [Int], Int, Bool
- monade ist typkonstruktorklasse * -> *    zb Maybe, []

## meine frage

ja also [] ist monade und [a] ist monoid

```haskell
instance Semigoup [a] where
    (<>) = (++)

instance Monoid [a] where
    mempty = []

## partielles maxlist mittels foldr

In [29]:
maxlist :: [Int] -> Int
maxlist = foldr max (-3000 )

maxlist (10:[1..3])

10

```haskell
foldr1 :: forall (t :: * -> *) a. Foldable t => (a -> a -> a) -> t a -> a
```
die idee von foldr1 ist, dass ich bei dem letzten element anfange und dem vorersten aufhöre. somit kann ich mir das element für die leere Liste sparen

In [28]:
maxlist :: [Int] -> Int
maxlist = foldr1 max

maxlist (10:[1..3])

Line 2: Use maximum
Found:
foldr1 max
Why not:
maximum

10

In [27]:
maxlist :: [Int] -> Int
maxlist = foldl1 max

maxlist (10:[1..3])

Line 2: Use maximum
Found:
foldl1 max
Why not:
maximum

10

# Faltung Wiederholung

als beispiel blattmarkierte bäume

In [30]:
data Tree a = Leaf a 
            | Tree a :+: Tree a

```haskell
(:+:) :: Tree a -> Tree a -> Tree a
Leaf  :: a -> Tree a
```

- was ich nun bei der faltung mache ist, ich ersetze jeden typkonstruktor mit einer funktion
- udn dann im tree selber ersetze ich das auch

In [35]:
foldTree :: (a -> b) -> (b -> b -> b) -> Tree a -> b
foldTree l _ (Leaf a)      = l a
foldTree l f (tl :+: tr) = foldTree l f tl `f` foldTree l f tr

In [37]:
maxInTree :: Ord a => Tree a -> a 
maxInTree = foldTree id max

In [52]:
testTreeinf :: Int -> Tree Int
testTreeinf n = Leaf n :+: testTreeinf (n+1)

testTree = Leaf 1 :+: (Leaf 3 :+: Leaf 5)

In [41]:
(!!!) :: Tree a -> String -> Tree a
(Leaf a) !!! "" = Leaf a
-- (tl :+: tr) !!! "" = (tl :+: tr)
t !!! "" = t
(tl :+: tr) !!! ('l':dirs) = tl  !!! dirs 
(tl :+: tr) !!! ('r':dirs) = tr  !!! dirs

# Funktionale Listen

## Show instanz für diesen Typen

In [43]:
:i Show

type Show :: * -> Constraint
class Show a where
  showsPrec :: Int -> a -> ShowS
  show :: a -> String
  showList :: [a] -> ShowS
  {-# MINIMAL showsPrec | show #-}
  	-- Defined in ‘GHC.Show’
instance forall k a (b :: k). Show a => Show (Const a b) -- Defined in ‘Data.Functor.Const’
instance (Show a, Show b) => Show (Either a b) -- Defined in ‘Data.Either’
instance Show a => Show (ZipList a) -- Defined in ‘Control.Applicative’
instance Show IHaskellSysIO.NewlineMode -- Defined in ‘GHC.IO.Handle.Types’
instance Show IHaskellSysIO.Newline -- Defined in ‘GHC.IO.Handle.Types’
instance Show IHaskellSysIO.Handle -- Defined in ‘GHC.IO.Handle.Types’
instance Show IHaskellSysIO.BufferMode -- Defined in ‘GHC.IO.Handle.Types’
instance Show IHaskellSysIO.HandlePosn -- Defined in ‘GHC.IO.Handle’
instance Show a => Show [a] -- Defined in ‘GHC.Show’
instance Show Word -- Defined in ‘GHC.Show’
instance Show a => Show (Solo a) -- Defined in ‘GHC.Show’
instance Show GHC.Types.RuntimeRep -- Defined in ‘

In [44]:
:i ShowS

type ShowS :: *
type ShowS = String -> String
  	-- Defined in ‘GHC.Show’

In [54]:
instance  Show a => Show (Tree a) where
    showsPrec p (Leaf x)    = showsPrec p x
    -- showsPrec p (tl :+: tr) = showsPrec p tl . (\s -> ":+:" ++ s) . showsPrec p tr
    showsPrec p (tl :+: tr) = showsPrec p tl . showString ":+:" . showsPrec p tr

testTree

1:+:3:+:5